This is the <a href="https://jupyter.org/">Jupyter Notebook</a>, an interactive coding and computation environment. For this lab, you do not have to write any code, you will only be running it. 

To use the notebook:
- "Shift + Enter" runs the code within the cell (so does the forward arrow button near the top of the document)
- You can alter variables and re-run cells
- If you want to start with a clean slate, restart the Kernel either by going to the top, clicking on Kernel: Restart, or by "esc + 00" (if you do this, you will need to re-run the following block of code before running any other cells in the notebook) 

This notebook uses code adapted from 

SimPEG
- Cockett, R., S. Kang, L.J. Heagy, A. Pidlisecky, D.W. Oldenburg (2015, in review), SimPEG: An open source framework for simulation and gradient based parameter estimation in geophysical applications. Computers and Geosciences

In [ ]:
from library.Mag import *
from SimPEG import PF, Utils, Mesh
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from scipy.interpolate import griddata
from scipy.spatial import cKDTree
from scipy.interpolate.interpnd import _ndim_coords_from_arrays
from scipy.sparse.linalg import bicgstab
from SimPEG.Utils import mkvc

%pylab inline

# How do we define direction of an earth magnetic field?

Earth magnetic field is a vector. To define a vector we need to choose a coordinate system. We use right-handed system: 
- X (Easting), 
- Y (Northing), and 
- Z (Up). 

Here we consider an earth magnetic field ($\vec{B_0}$), of which intensity is one. To define this unit vector, we use inclinatino and declination:
- Declination: An angle from geographic North (Ng) (positive clockwise)
- Inclination: Vertical angle from the N-E plane (positive down)

<img src="https://github.com/geoscixyz/gpgLabs/raw/master/figures/Mag/earthfield.png?raw=true" style="width: 60%; height: 60%"> </img>

# What's data: total field anomaly

We consider a typical form of magnetic data. To illustrate this we consider an suceptible object embedded in the earth. 
Based upon the earth magnetic field ($\vec{B}_0$), this object will generate anomalous magnetic field ($\vec{B}_A$). We define an unit vector $\hat{B}_0$ for the earth field as 
$$ \hat{B}_0 = \frac{\vec{B}_0}{|\vec{B}_0|}$$ 
We measure both earth and anomalous magnetic field such that

$$ \vec{B} = \vec{B}_0 + \vec{B}_A$$

Total field anomaly, $\triangle \vec{B}$ can be defined as

$$  |\triangle \vec{B}| = |\vec{B}|-|\vec{B}_E| $$ 

If $|\vec{B}|\ll|\vec{B}_E|$, then that is total field anomaly $\triangle \vec{B}$ is the projection of the anomalous field onto the direction of the earth field:

$$ |\triangle \vec{B}| \simeq \vec{B}_A \cdot \hat{B}_0=|\vec{B}_A|cos\theta$$ 

<img src="https://github.com/geoscixyz/gpgLabs/raw/master/figures/Mag/totalfieldanomaly.png?raw=true" style="width: 50%; height: 50%">

In [ ]:
#Input parameters
# fileName = 'http://github.com/geoscixyz/gpgLabs/raw/master/assets/Mag/data/DO27_TMI.dat'
xyzd = np.genfromtxt('C:\\Dianne\\Data\\Public\\GBC_Search\\Phase2\\Mag\\GBCR2018-02-GDB-Magnetics\\Search_reduced_columns_DwnS100m.dat')

B = np.r_[60308, 83.8, 25.4]
survey = Mag.createMagSurvey(np.c_[xyzd[:,4], xyzd[:,5], xyzd[:,2], xyzd[:,3]], B)
# View the data and chose a profile
param = Simulator.ViewMagSurvey2D(survey)
display(param)

Data griddind
=======

Before proceeding with the Fourier filters, we need to grid the data.
Several papers comparing the different interpolation methods.

Minimum curvature is often the best for potential fields

In [ ]:
from scipy.sparse.linalg import bicgstab

def minCurvatureInterp(xyzd, gridSize = 100):
    
    # Define a new grid based on data extent
    xmin, xmax = xyzd[:,0].min(), xyzd[:,0].max()
    ymin, ymax = xyzd[:,1].min(), xyzd[:,1].max()

    nCx = int((xmax-xmin)/gridSize)
    nCy = int((ymax-ymin)/gridSize)

    vecCx = xmin+np.cumsum(np.ones(nCx) * gridSize)
    vecCy = ymin+np.cumsum(np.ones(nCy) * gridSize)

    gridCx, gridCy = np.meshgrid(vecCx, vecCy)

    ndat = xyzd.shape[0]
    nC = int(nCx*nCy)

    A = np.zeros((ndat, ndat))
    for i in range(ndat):

        r = (xyzd[i,0] - xyzd[:,0])**2. + (xyzd[i,1] - xyzd[:,1])**2.
        A[i, :] = r.T * (np.log((r.T + 1e-8)**0.5) - 1.)

    # Solve system for the weights
    w = bicgstab(A, xyzd[:,-1], tol=1e-6)

    # Compute new solution
    # Reformat the line data locations but skip every n points for test

    xx = mkvc(gridCx)
    yy = mkvc(gridCy)
    dMinCurv = np.zeros_like(xx)
    
    # We can parallelize this part later
    for i in range(nC):

        r = (xx[i] - xyzd[:,0])**2. + (yy[i] - xyzd[:,1])**2.
        dMinCurv[i] = np.sum( w[0] * r.T * ( np.log( (r.T + 1e-8)**0.5 ) - 1. ))

    return gridCx, gridCy, dMinCurv.reshape(gridCx.shape, order='F')

In [ ]:
X, Y, dMinCurv = minCurvatureInterp(xyzd, gridSize = 20)

Data plot
=====

Next step we want to plot the data

In [ ]:
plt.figure()
axs = plt.subplot()
plt.contourf(X, Y, dMinCurv,20, cmap='RdBu_r')
plt.contour(X, Y, dMinCurv,20,colors='k')
plt.scatter(xyzd[:,0], xyzd[:,1],2, color='k')
axs.set_aspect('equal')
plt.title('Boring 2D plot')
plt.show()

Advanced plot
=======

We can do better by adding "relief" and sun shading

In [ ]:
# from matplotlib.colors import LightSource
from scipy.interpolate import griddata
from scipy.spatial import cKDTree
from scipy.interpolate.interpnd import _ndim_coords_from_arrays
from matplotlib.colors import LightSource

def plotDataHillside(x, y, z, axs=None, fill=True, contour=0, vmin=None, vmax=None,
                     clabel=True, cmap = 'RdBu_r', ve=1., alpha=1., alphaHS=1.,
                     distMax=1000, midpoint=None, azdeg=315, altdeg=45):
    
    ls = LightSource(azdeg=azdeg, altdeg=altdeg)
    
    if x.ndim == 1:
        # Create grid of points
        x = np.linspace(x.min(), x.max(), 1000)
        y = np.linspace(y.min(), y.max(), 1000)

        X, Y = np.meshgrid(x, y)

        # Interpolate
        d_grid = griddata(np.c_[x,y], z, (X, Y), method='cubic')
    
    else:
        
        X, Y, d_grid = x, y, z
        
    # Remove points beyond treshold
#     tree = cKDTree(np.c_[x,y])
#     xi = _ndim_coords_from_arrays((X, Y), ndim=2)
#     dists, indexes = tree.query(xi)

#     # Copy original result but mask missing values with NaNs
#     d_grid[dists > distMax] = np.nan


    class MidPointNorm(Normalize):    
        def __init__(self, midpoint=midpoint, vmin=None, vmax=None, clip=False):
            Normalize.__init__(self,vmin, vmax, clip)
            self.midpoint = midpoint

        def __call__(self, value, clip=None):
            if clip is None:
                clip = self.clip

            if self.midpoint is None:
                midpoint = np.mean(value)
                
            result, is_scalar = self.process_value(value)

            self.autoscale_None(result)
            vmin, vmax = self.vmin, self.vmax

            if not (vmin < midpoint < vmax):
                raise ValueError("midpoint must be between maxvalue and minvalue.")       
            elif vmin == vmax:
                result.fill(0) # Or should it be all masked? Or 0.5?
            elif vmin > vmax:
                raise ValueError("maxvalue must be bigger than minvalue")
            else:
                vmin = float(vmin)
                vmax = float(vmax)
                if clip:
                    mask = ma.getmask(result)
                    result = ma.array(np.clip(result.filled(vmax), vmin, vmax),
                                      mask=mask)

                # ma division is very slow; we can take a shortcut
                resdat = result.data

                #First scale to -1 to 1 range, than to from 0 to 1.
                resdat -= midpoint            
                resdat[resdat>0] /= abs(vmax - midpoint)            
                resdat[resdat<0] /= abs(vmin - midpoint)

                resdat /= 2.
                resdat += 0.5
                result = ma.array(resdat, mask=result.mask, copy=False)                

            if is_scalar:
                result = result[0]            
            return result

        def inverse(self, value):
            if not self.scaled():
                raise ValueError("Not invertible until scaled")
            vmin, vmax, midpoint = self.vmin, self.vmax, self.midpoint

            if cbook.iterable(value):
                val = ma.asarray(value)
                val = 2 * (val-0.5)  
                val[val>0]  *= abs(vmax - midpoint)
                val[val<0] *= abs(vmin - midpoint)
                val += midpoint
                return val
            else:
                val = 2 * (val - 0.5)
                if val < 0: 
                    return  val*abs(vmin-midpoint) + midpoint
                else:
                    return  val*abs(vmax-midpoint) + midpoint

    im, CS= [], []
    if axs is None:
        axs = plt.subplot()
        
    if fill:
        extent = x.min(), x.max(), y.min(), y.max()
        im = axs.contourf(X, Y, d_grid, 50, vmin=vmin, vmax=vmax, clim=[vmin,vmax],
                          cmap=cmap, norm=MidPointNorm(midpoint=midpoint), alpha=alpha)
        
        axs.imshow(ls.hillshade(d_grid, vert_exag=ve, dx=1., dy=1.),
                        cmap='gray', alpha=alphaHS,
                        extent=extent, origin='lower')

    if contour > 0:
        CS = axs.contour(X, Y, d_grid, int(contour), colors='k', vmin=vmin, vmax=vmax, linewidths=0.5)
        
        if clabel:
            plt.clabel(CS, inline=1, fontsize=10, fmt='%i')
    return im, CS 

# DO18 = np.loadtxt('C:\\Users\DominiqueFournier\\ownCloud\\TKC_CaseStudy\\Modelling\\GIS\\Harder_Outlines\\Harder2008_DO18_Outline.dat')
# DO27 = np.loadtxt('C:\\Users\DominiqueFournier\\ownCloud\\TKC_CaseStudy\\Modelling\\GIS\\Harder_Outlines\\Harder2008_DO27_Outline.dat')


In [ ]:
import ipywidgets as widgets

def dataViewer(X,Y,dataGrid):
      
    def plotData(azdeg,altdeg,alpha,alphaHS,contour):
        fig = plt.figure(figsize=(8,6))
        axs= plt.subplot()

        # Read the data
#         vmin, vmax = -25, 150
        cmap = plt.cm.gist_earth

        # Add shading
        im, CS = plotDataHillside(X, Y,dataGrid ,
                                  axs=axs, cmap='RdBu_r', 
                                  clabel=False, contour=contour,
                                  alpha=alpha, alphaHS=alphaHS, ve=1., azdeg=azdeg, altdeg=altdeg)

        # Add points at the survey locations
        plt.scatter(xyzd[:,0], xyzd[:,1], s=2, c='k')

        # Set limits and labels
#         axs.set_xlim([xmin,xmax])
#         axs.set_ylim([ymin,ymax])
        # axs.set_xticklabels([556750,558000, 559250], size=14)
        # axs.set_xticks([556750,558000, 559250])
        # axs.set_yticklabels([7133000,7134000, 7135000], size=14, rotation=90, va='center')
        plt.colorbar(im)
        # axs.set_yticks([7133000,7134000, 7135000])
        axs.set_xlabel("Easting (m)", size=14)
        axs.set_ylabel("Northing (m)", size=14)
        axs.grid('on', color='k', linestyle='--')
        plt.show()
        # Add colorbar
        # pos = axs.get_position() # Get the position of previous axes
        # axbar = plt.axes([pos.x0+.25, pos.y0+0.2,  pos.width*0.75, pos.height*.5]) # Create a new axes and reshape
        # cbar = plt.colorbar(im, orientation='vertical',format='%.3f') # Add a colorbar using the color definition of previous
        # cbar.set_ticks(np.linspace(-25,300,6)) # Change the tick position
        # cbar.set_ticklabels(np.round(np.linspace(-25,300,6))) # Change the tick labels
        # axbar.text(pos.x0+1., pos.y0+.75,'SI', size=14) # Add units and move it above
        # axbar.axis('off') # Only keep the colorbar, remove the rest

    out = widgets.interactive(plotData,
                              azdeg=widgets.FloatSlider(min=0, max=360, step=5, value=0, continuous_update=False),
                              altdeg=widgets.FloatSlider(min=0, max=90, step=5, value=45, continuous_update=False),
                              alpha=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.2, continuous_update=False),
                              alphaHS=widgets.FloatSlider(min=0, max=1, step=0.1, value=1.0, continuous_update=False),
                              contour=widgets.FloatSlider(min=0, max=20, step=1, value=10, continuous_update=False))
    return out

view = dataViewer(X, Y, dMinCurv)
display(view)

Fourier Filters
========

Next we want to look at data filters

1- Total derivative

2- 1th Vertical derivative

In [ ]:
def padTapperGrid(d_grid):
        ## Figure out padding width
    padx = int(np.floor(d_grid.shape[1]))
    pady = int(np.floor(d_grid.shape[0]))

    # Add paddings
    dpad = np.c_[np.fliplr(d_grid[:,0:padx]),
                 d_grid,
                 np.fliplr(d_grid[:,-padx:])
                ]

    dpad = np.r_[np.flipud(dpad[0:pady,:]),
                 dpad,
                 np.flipud(dpad[-pady:,:])
                ]

    dx = (1)/(dpad.shape[1]-1)
    dy = (1)/(dpad.shape[0]-1)

    kx = np.fft.fftfreq(dpad.shape[1], dx)
    ky = np.fft.fftfreq(dpad.shape[0], dy)

    Ky, Kx = numpy.meshgrid(ky, kx)

    # Tapper the paddings
    rampx = -np.cos(np.pi*np.asarray(range(padx))/padx)
    rampx = np.r_[rampx,np.ones(d_grid.shape[1]),-rampx]/2. + 0.5
    # tapperx,_ = meshgrid(rampx,np.ones(dpad.shape[1]))
    # tapperx[padx:-padx,:] = 1.

    rampy = -np.cos(np.pi*np.asarray(range(pady))/pady)
    rampy = np.r_[rampy,np.ones(d_grid.shape[0]),-rampy]/2. +0.5
    tapperx, tappery = meshgrid(rampx, rampy)

    gridOut = tapperx*tappery*dpad
    return padx, pady, Kx.T, Ky.T, gridOut


def getFFTgrid(d_grid):

    npadx, npady, Kx, Ky, dpad_taped = padTapperGrid(d_grid)
    # Now that we have a grid we can run the FFT
    FS = np.fft.fft2(dpad_taped)
    
    return npadx, npady, Kx, Ky, FS

def gridFilers(d_grid, filterType='1VD'):

    
    padx, pady, Kx, Ky, FFTgrid = getFFTgrid(d_grid)
    
    FHxD = (Kx*1j)*FFTgrid
    FHyD = (Ky*1j)*FFTgrid
    
    
    if filterType == 'FHDx':
        
        fhxd_pad = np.fft.ifft2(FHxD)
        gridOut = np.real(fhxd_pad[pady:-pady, padx:-padx])
    
    if filterType == 'FHDy':
        
        fhyd_pad = np.fft.ifft2(FHyD)
        gridOut = np.real(fhyd_pad[pady:-pady, padx:-padx])

    if filterType == '1VD':
        FHzD = FFTgrid*np.sqrt(Kx**2 + Ky**2)
        fhzd_pad = np.fft.ifft2(FHzD)
        gridOut = np.real(fhzd_pad[pady:-pady, padx:-padx])

    if filterType == 'THD':
        fhxd_pad = np.fft.ifft2(FHxD)
        fhxd = np.real(fhxd_pad[pady:-pady, padx:-padx])
        
        fhyd_pad = np.fft.ifft2(FHyD)
        fhyd = np.real(fhyd_pad[pady:-pady, padx:-padx])
                
        gridOut = np.sqrt(fhxd**2 + fhyd**2)
        
    if filterType == 'TiltAngle':
        
        FHzD = FFTgrid*np.sqrt(Kx**2 + Ky**2)
        fhzd_pad = np.fft.ifft2(FHzD)
        fhzd = np.real(fhzd_pad[pady:-pady, padx:-padx])
        gridOut = np.arctan2(fhzd, horiz_deriv)


    return gridOut


filteredData = gridFilers(dMinCurv, filterType='1VD')

# Plot again fancy

view = dataViewer(X, Y, filteredData)
display(view)

# ax2.set_title('Horizontal Derivative - X')

# plt.subplot(1,3,2)
# plt.imshow(tappery*tapperx,origin='lower')

# plt.subplot(1,3,3)
# plt.imshow(dpad_taped,origin='lower')

In [ ]:
from skimage.measure import compare_ssim, find_contours
from skimage.feature import canny

def findEdges(dataGrid):
    
    def plotData(sigma,threshold):

        plt.figure()
        plt.subplot()

        edges = canny(
            filteredData, 
            sigma=sigma, 
            low_threshold=threshold, 
            use_quantiles=True)
        
        plt.contourf(X,Y,edges,1)
        
        plt.show()

    out = widgets.interactive(plotData,
                                  sigma=widgets.FloatSlider(min=0, max=4, step=0.1, value=1, continuous_update=False),
                                  threshold=widgets.FloatSlider(min=0, max=4, step=0.1, value=0.01, continuous_update=False))
                                  
    return out
                              
view = findEdges(filteredData)
                              
display(view)
                              

In [ ]:
import skimage as sk

edges = canny(
            filteredData, 
            sigma=3.1, 
            low_threshold=0.1, 
            use_quantiles=True)

lines = sk.transform.hough_line(edges)

In [ ]:
lines

In [ ]:
# Define the parametric model interactively
model = Simulator.ViewPrism(param.result)
display(model)

# Magnetic applet
Based on the prism that you made above, below Magnetic applet computes magnetic field at receiver locations, and provide both 2D map (left) and profile line (right). 

For the prism, you can alter:
- sus: susceptibility of the prism

Parameters for the earth field are:
- Einc: inclination of the earth field (degree)
- Edec: declination of the earth field (degree)
- Bigrf: intensity of the earth field (nT)

For data, you can view:
- tf: total field anomaly,  
- bx :x-component, 
- by :y-component, 
- bz :z-component

You can simulate and view remanent magnetization effect with parameters:
- irt: "induced", "remanent", or "total"
- Q: Koenigsberger ratio ($\frac{M_{rem}}{M_{ind}}$)
- rinc: inclination of the remanent magnetization (degree)
- rdec: declination of the remanent magnetization (degree)


In [ ]:
plotwidget = Simulator.PFSimulator(model, param)
display(plotwidget)